In [10]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf

import pandas as pd

In [16]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [13]:
query = f"TRUNCATE product_characteristics ;"
cursor.execute(query)


In [17]:
query = f'ALTER TABLE product_img ADD COLUMN base64 text'
cursor.execute(query)

# Fetch the rows
#rows = cursor.fetchall()
#rows

In [18]:
conn.commit()

In [ ]:
query = f"SELECT product_characteristics.id, Brand_id, Detail, Brand, Price, Link FROM product_characteristics INNER JOIN Products ON product_characteristics.Brand_id = Products.Brand_Prod_id ;"
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()
#rows

In [9]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [ ]:
query = f"SELECT Brand_id, image_link FROM product_img ;"
cursor.execute(query)

# Fetch the rows
rows_img = cursor.fetchall()
#rows

In [ ]:
SQLf.sql_db_functions.close_connection_db(conn=conn,cursor=cursor)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(rows ,columns= ['id', 'Brand_id', 'Detail', 'Brand', 'Price', 'Link'])
df.head()

In [ ]:
df_img = pd.DataFrame(rows_img ,columns= ['Brand_id','image_link'])
df_img.head()

### Organize the Details

In [ ]:
df[['Brand_id','Detail']].iloc[0].Detail